# Segmenting and Clustering Neighborhoods in Toronto

## Part 1 - Read the Toronto Neightborhood data from wikipedia

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Wikipedia page to load
wiki_page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M";

# Get the html page
wiki_data = requests.get(wiki_page);

# response code 200 means the data was read OK
print(wiki_data)

<Response [200]>


Parse the html page using beautiful_soup.

In [3]:
soup = BeautifulSoup(wiki_data.text);

Use the hint code provided to scrape the table data and skip unassigned postal codes.

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)


Convert the list to a DataFrame

In [5]:
df=pd.DataFrame(table_contents);

Clean up the Borough names

In [6]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [7]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
Verify that no neighborhoods are not assigned

In [8]:
df.loc[df['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


In [9]:
df.shape

(103, 3)